

# Lecture 12. Keras Functional API

## Sequential API
- model.add를 통해 layer를 순차적으로 추가 
    <img src="figures/sequential.PNG" width="20%">

## Functional API
- 함수의 input, output 형태로 layer 간의 관계를 정의 
- 다양한 네크워크 구조 생성 가능 

#### Multiple inputs model

<img src="figures/multi_input.PNG" width="35%">

#### Multiple outputs model
<img src="figures/multi_output.PNG" width="35%">

#### Model with shared layers
<img src="figures/concat.PNG" width="45%">

<br><br>
#### Model construction with function API
1. Input 정의
    - Sequential model과 다르게 input layer 정의가 필요

2. Layer를 연결 
    - 새로운 layer를 생성할 때 이전 layer의 output을 input으로 입력

3. Model 생성 
    - 최종 모델의 input과 output을 지정 

## Example 1

In [1]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, LSTM, Embedding, concatenate
from keras.utils import plot_model

from IPython.display import Image

Using TensorFlow backend.


#### Sequential API

In [2]:
model=Sequential()
model.add(Dense(2, input_shape = (2,)))
model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________



#### Functional API

In [3]:
visible = Input(shape=(2,))
hidden = Dense(2)(visible)
model = Model(inputs = visible, outputs = hidden)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


## Example 2

#### Sequential API 

In [4]:
model = Sequential()
model.add(LSTM(10, input_shape = (100,1)))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10)                480       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________



#### Functional API

## Example 3

In [5]:
from keras.layers import MaxPooling2D, Flatten, Conv2D
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 175232)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               89719296  
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 513       
Total params: 89,720,705
Trainable params: 89,720,705
Non-trainable params: 0
_________________________________________________________________


## Example 4: Multiple inputs
- Question and Answering model 
    - Input 1: 신문 등과 같이 대답을 위한 정보가 포함된 articles 
    - Input 2: 질문 text
    
- 각 input을 LSTM을 통해 학습한 후 두 output layer를 가로로 연결(concatenate)
- Dense layer를 통해 answer를 생성하기 위한 output layer로 연결
<img src="figures/multi_input2.PNG" width="30%">

In [6]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# Input 1
text_input = Input(shape=(None,), name='text')
embedded_text = Embedding(text_vocabulary_size, 64, input_length = 10)(text_input)
encoded_text = LSTM(32)(embedded_text)

# Input 2
question_input = Input(shape=(None,), name='question')
embedded_question = Embedding(question_vocabulary_size, 32, input_length = 5)(question_input)
encoded_question = LSTM(16)(embedded_question)

# Concatenation
concatenated = concatenate([encoded_text, encoded_question], axis=-1)
answer = Dense(answer_vocabulary_size, activation='softmax')(concatenated)

# Define a model
model = Model([text_input, question_input], answer)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 10, 64)       640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 5, 32)        320000      question[0][0]                   
____________________________________________________________________________________________

- Model을 적합할 때 두 개의 input data를 list 혹은 dictionary 형태(input의 name이 지정되어 있을 때만)로 입력 

In [7]:
## DO NOT RUN 
model.fit([text, question], answers, epochs=10, batch_size=128)
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

NameError: ignored

<br><br><br>

## Example 5: Multiple outputs

- Social media에 포스팅한 text를 사용하여 사용자의 여러 특성을 예측
    - Input: social media posts
    - Output 1: Age
    - Output 2: Income
    - Output 3: Gender


<img src="figures/multi_output2.PNG" width="30%">

In [8]:
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = Embedding(vocabulary_size, 256)(posts_input)
x = Conv1D(128, 5, activation='relu')(embedded_posts)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Dense(128, activation='relu')(x)

age_prediction = Dense(1, name='age')(x)# Output 1
income_prediction = Dense(num_income_groups, activation='softmax', name='income')(x)# Output 2
gender_prediction = Dense(1, activation='sigmoid', name='gender')(x)# Output 3

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])# Define a model

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 128)    163968      embedding_3[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 128)    0           conv1d_1[0][0]                   
____________________________________________________________________________________________

- Model compile 단계에서 각 output에 대한 loss function과 weight를 지정
- Model fit 단계에서 output data를 list 형태로 입력 

In [9]:
# DO NOT RUN

model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.],
             metrics = {'gender': auc})
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

NameError: ignored